In [1]:
!unzip /content/UDD.v1i.yolov5pytorch.zip

Archive:  /content/UDD.v1i.yolov5pytorch.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
 extracting: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/0009_jpg.rf.468557fdf3da784d6ccded16e9366f72.jpg  
 extracting: test/images/0023_jpg.rf.600eadd57cc9d0bfa824c092d8dce7ed.jpg  
 extracting: test/images/0028_jpg.rf.317d20c5c8da76ebba15cf6e5fb41c0d.jpg  
 extracting: test/images/0042_jpg.rf.5c6e382accb6c88a9dfa71eaf032efd6.jpg  
 extracting: test/images/0053_jpg.rf.09c5a0f5af288f9426616259b0307fae.jpg  
 extracting: test/images/0083_jpg.rf.cee780cf39c6d3b7157d81217fbbdb00.jpg  
 extracting: test/images/0096_jpg.rf.ab657cc8420719de58193cdffe8265c3.jpg  
 extracting: test/images/0099_jpg.rf.cad36a20a40ebb0de2378db1214eeef1.jpg  
 extracting: test/images/0124_jpg.rf.f3228afe3b02618ff98e8fe373875908.jpg  
 extracting: test/images/0189_jpg.rf.3e1771185e2374613a8de8ef804451f8.jpg  
 extracting: test/images/0204_jpg.rf

In [2]:
import os, time, random
import numpy as np
import pandas as pd
import cv2, torch
from tqdm.auto import tqdm
import shutil as sh

from IPython.display import Image, clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
id_list = []
labels_list = []
path_image_list = []
train = pd.DataFrame()

directory = '/content/train/labels'
for filename in os.listdir(directory):
    print(filename)
    id_list.append(filename[:4])
    path_image_list.append(f'{filename[:-4]}.jpg')
    object_list = []
    archive = open(f'{directory}/{filename}')
    for line in archive.readlines():
        l = [float(i) for i in line.split(' ')]
        object_list.append(l)

    labels_list.append(object_list)

train['image_id'] = id_list
train['labels'] = labels_list
train['path_images'] = path_image_list

train.sort_values(by=['image_id'], inplace=True)

0073_jpg.rf.8975ac54732b625a656d75f27da1b2a7.txt
0996_jpg.rf.ed2a19c6fbf0e1bec958b03474e40bf3.txt
16968_jpg.rf.5917e3c8e56f603c1e553662db8eb50b.txt
0048_jpg.rf.46acfd74a541310b4f38a3535e1782d2.txt
0230_jpg.rf.be61e8d4b8b1100fbced36c780e7d8d3.txt
0620_jpg.rf.440890e4ff7523a8cd27a476df06a620.txt
0788_jpg.rf.36a16df372002f98003fa132520755cb.txt
17141_jpg.rf.10a772d4881997e04d2e46d190c7b9b0.txt
0312_jpg.rf.fa648d471d98a6172aad5940e1ec8a22.txt
1426_jpg.rf.35f6fdf56624a4b2a9fe200f43ab0270.txt
17484_jpg.rf.706bc3bbfcd6a5c64bd63e617c627823.txt
1478_jpg.rf.b66d6b6689ab111ccdffc949a775224c.txt
19047_jpg.rf.ccce99054d7230b6ccc4f82b96fd3d09.txt
1595_jpg.rf.a3ee7bf3619c07af2f60d1b400ad8c8b.txt
0217_jpg.rf.574d260580c3ed65bc32f6b82d671d55.txt
0888_jpg.rf.e6e157e0a01979c7f7784ab8ee739b4e.txt
1223_jpg.rf.e7e7b468758f8f76a51165ddcd8d133f.txt
0209_jpg.rf.b72c189a7ba636ef5032b71862f3580b.txt
0249_jpg.rf.c741b7cc650c17a5ef5d3190d50d6cb4.txt
2042_jpg.rf.15447d7cb50b7d076da59873fb07f913.txt
2218_jpg.rf.5c3a

In [4]:
train_csv = pd.read_csv('/content/_annotations.csv')
train_csv.sort_values(by=['filename'], inplace=True)
train_csv

,filename,width,height,class,xmin,ymin,xmax,ymax
6054,0000_jpg.rf.424c6f7925c1060f4c441a7c1d09bf3f.jpg,640,640,seaurchin,336,119,376,180
6052,0000_jpg.rf.424c6f7925c1060f4c441a7c1d09bf3f.jpg,640,640,seaurchin,85,412,141,492
6053,0000_jpg.rf.424c6f7925c1060f4c441a7c1d09bf3f.jpg,640,640,seaurchin,157,111,197,173
11164,0003_jpg.rf.5ce322d113737a4735016b246fd4caff.jpg,640,640,seaurchin,198,130,226,173
11165,0003_jpg.rf.5ce322d113737a4735016b246fd4caff.jpg,640,640,seaurchin,58,18,86,59
...,...,...,...,...,...,...,...,...
2525,2226_jpg.rf.997e34c9f340dd0345966fe28a8007b7.jpg,640,640,seaurchin,204,158,284,302
2526,2226_jpg.rf.997e34c9f340dd0345966fe28a8007b7.jpg,640,640,seaurchin,241,281,322,411
2529,2226_jpg.rf.997e34c9f340dd0345966fe28a8007b7.jpg,640,640,seaurchin,2,82,93,316
2527,2226_jpg.rf.997e34c9f340dd0345966fe28a8007b7.jpg,640,640,seaurchin,350,324,403,419


## Load Dataset

In [9]:
img_h, img_w, num_channels = (640, 640, 3)
#train['image_id'] = train['image_id'].apply(lambda x: x.split('.')[0])
train_csv['x_center'] = (train_csv['xmin'] + train_csv['xmax'])/2
train_csv['y_center'] = (train_csv['ymin'] + train_csv['ymax'])/2
train_csv['w'] = train_csv['xmax'] - train_csv['xmin']
train_csv['h'] = train_csv['ymax'] - train_csv['ymin']
train_csv['x_center'] = train_csv['x_center']/img_w
train_csv['w'] = train_csv['w']/img_w
train_csv['y_center'] = train_csv['y_center']/img_h
train_csv['h'] = train_csv['h']/img_h
train_csv.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,x_center,y_center,w,h
6054,0000_jpg.rf.424c6f7925c1060f4c441a7c1d09bf3f.jpg,640,640,seaurchin,336,119,376,180,0.556250,0.233594,0.06250,0.095312
6052,0000_jpg.rf.424c6f7925c1060f4c441a7c1d09bf3f.jpg,640,640,seaurchin,85,412,141,492,0.176563,0.706250,0.08750,0.125000
6053,0000_jpg.rf.424c6f7925c1060f4c441a7c1d09bf3f.jpg,640,640,seaurchin,157,111,197,173,0.276562,0.221875,0.06250,0.096875
11164,0003_jpg.rf.5ce322d113737a4735016b246fd4caff.jpg,640,640,seaurchin,198,130,226,173,0.331250,0.236719,0.04375,0.067187
11165,0003_jpg.rf.5ce322d113737a4735016b246fd4caff.jpg,640,640,seaurchin,58,18,86,59,0.112500,0.060156,0.04375,0.064062


In [10]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.4/709.4 kB 13.3 MB/s eta 0:00:00


In [11]:
# Create .yaml file
import yaml

data_yaml = dict(
    train = '/content/train/images',
    val = '../content/test/images',
    nc = 3,
    names = ['scalop', 'seacucumber', 'seaurchin']
)

# Note that I am creating the file in the yolov5/data/ directory.
with open('data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

In [12]:
import os
import sys
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

results = model.train(data="/content/data.yaml", epochs=50, pretrained=True, iou=0.5, visualize=True, patience=0)  # train the model
results = model.val()

100%|██████████| 6.23M/6.23M [00:00<00:00, 286MB/s]


Ultralytics YOLOv8.1.8 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/data.yaml, epochs=50, time=None, patience=0, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=True, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

100%|██████████| 755k/755k [00:00<00:00, 47.4MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/train/labels... 1892 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1892/1892 [00:02<00:00, 722.03it/s]


...


50 epochs completed in 0.910 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.8 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.15it/s]


                   all        111        757      0.695      0.643      0.701      0.363
                scalop        111         14      0.697      0.571      0.677      0.362
           seacucumber        111         56      0.558      0.518      0.541      0.296
             seaurchin        111        687       0.83       0.84      0.886      0.432
Speed: 0.2ms preprocess, 3.2ms inference, 0.0ms loss, 5.0ms postprocess per image
Results saved to runs/detect/train
Ultralytics YOLOv8.1.8 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/test/labels.cache... 111 images, 0 backgrounds, 0 corrupt: 100%|██████████| 111/111 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:07<00:00,  1.13s/it]


                   all        111        757      0.705      0.642      0.698      0.366
                scalop        111         14      0.712      0.571      0.667      0.373
           seacucumber        111         56      0.568      0.518      0.541      0.291
             seaurchin        111        687      0.834      0.835      0.887      0.433
Speed: 0.5ms preprocess, 7.9ms inference, 0.0ms loss, 6.4ms postprocess per image
Results saved to runs/detect/train2


In [72]:
#  inference on testing dataset
!yolo task=detect mode=predict model='/content/runs/detect/train/weights/best.pt' save_txt=True conf=0.25 save_conf=True source='/content/test/images/'

Ultralytics YOLOv8.1.8 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs

image 1/111 /content/test/images/0009_jpg.rf.468557fdf3da784d6ccded16e9366f72.jpg: 640x640 1 seaurchin, 10.8ms
image 2/111 /content/test/images/0023_jpg.rf.600eadd57cc9d0bfa824c092d8dce7ed.jpg: 640x640 6 seaurchins, 8.9ms
image 3/111 /content/test/images/0028_jpg.rf.317d20c5c8da76ebba15cf6e5fb41c0d.jpg: 640x640 29 seaurchins, 7.2ms
image 4/111 /content/test/images/0042_jpg.rf.5c6e382accb6c88a9dfa71eaf032efd6.jpg: 640x640 7 seaurchins, 7.2ms
image 5/111 /content/test/images/0053_jpg.rf.09c5a0f5af288f9426616259b0307fae.jpg: 640x640 1 seacucumber, 5 seaurchins, 7.2ms
image 6/111 /content/test/images/0083_jpg.rf.cee780cf39c6d3b7157d81217fbbdb00.jpg: 640x640 2 seaurchins, 7.2ms
image 7/111 /content/test/images/0096_jpg.rf.ab657cc8420719de58193cdffe8265c3.jpg: 640x640 4 seaurchins, 7.2ms
image 8/111 /content/test/images/0099_jpg